In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import display, Image

In [13]:
from langgraph.prebuilt import  ToolNode, tools_condition

In [14]:
from langgraph.prebuilt import  ToolNode, tools_condition
from langchain_core.tools import tool
import requests

In [15]:


load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=api_key)

In [100]:
from typing import TypedDict, Literal, Optional
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage


    
    
from typing import TypedDict, Optional, List
from langchain_core.messages import HumanMessage, BaseMessage

class AgentState(TypedDict):
    status: str
    user_id: Optional[str]
    messages: List[BaseMessage]

def default_agent_state() -> AgentState:
    return {
        "status": "unregistered",
        "user_id": None,
        "messages": []
    }    


In [101]:
prompt_categorizer = """

your are a catgorizer you have to categorize  user either register or unregister
if user status is unregistered  categorize as unregistered
if user status is registered  categorize as registered
return only either "registered" or "unregistered"

"""

In [102]:


def categorizer(state: AgentState) -> AgentState:
    last_message = state["messages"][-1].content.lower()
    if "book" in last_message or "appointment" in last_message:
        if state["status"] == "unregistered":
            print("User wants to book but is unregistered.")
        else:
            print("User wants to book and is already registered.")
    return state

    




In [103]:
from langchain_core.tools import tool
import requests

FASTAPI_BASE_URL = "http://127.0.0.1:8000"

@tool
def call_register_user(_: str = "") -> dict:
    """Call the FastAPI endpoint to register a new user."""
    try:
        response = requests.post(f"{FASTAPI_BASE_URL}/register-user")
        if response.status_code == 200:
            data = response.json()
            print("✅ create account tool on go")
            print("✅ User registered via FastAPI")
            print(data["user_id"],  data["status"])
            return {"user_id": data["user_id"], "status": data["status"]}
        else:
            return {"error": "Failed to register user"}
    except Exception as e:
        return {"error": str(e)}

@tool
def call_confirm_registration(user_id: str) -> dict:
    """Call the FastAPI endpoint to confirm if user's account is created."""
    try:
        print(user_id)
        response = requests.get(f"{FASTAPI_BASE_URL}/confirm_registration", params={"user_id": user_id})
        if response.status_code == 200:
            data = response.json()
            print('the confirmation tools working')
            return {"user_id": user_id, "account_created": data.get("account_created", False)}
        else:
            return {"error": "Failed to confirm registration"}
    except Exception as e:
        return {"error": str(e)}


    
    

In [104]:
import csv

def check_user_id_in_csv(user_id: str, file_path=r"E:\Gen_AI2\lang_graph\prec\internship_task\task1_unregisterd_agent\new_with_fastapi\data.csv") -> bool:
    try:
        with open(file_path, mode='r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row["lead_id"] == user_id and row["status"] == "secured":
                    return True
    except Exception as e:
        print(f"Error reading CSV: {e}")
    return False


In [105]:
from langchain_core.tools import tool

tools = [call_register_user, call_confirm_registration]
llm_with_tools = llm.bind_tools(tools)


Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [106]:

prompt_unregister = """ 
Your are a customer support agent your job is to assist user on basic support

1. Friendly Introduction & Consent Request      
       
    - If the user says simple question like what is offiec timing or question about just inquery:
       a) Politely respond:  like offiec time are 2pm to 11pm


2.If the User is unregister:
    - user ask question like i want to book ticket , leads, or rise support tickets
    so if user is not register  these feature are not allowed
    -ask user a question do you want to register so you can access this features.
    if user response with yes so call a 
    response user with ' i have shared a link "https:google.com"  for registration' share this link with user

3.- If the user says something like:
  - "I have submitted the form"
  - "I completed the registration"
  - Or anything indicating completion

very very important    ask user to provide his user_id  so you can confirm

  Then:
  - 🔧 **You must call the tool** `call_confirm_registration` with the appropriate `user_id`.
  - Then wait for the tool result to confirm registration.
  must call this tool

- If the response confirms registration, route the user to the `registered` node.

---
    
   
-if user status is registered:
   then redirect it to registerd node
   
   

4. Communication Style
   - Be friendly, approachable, and polite throughout the interaction.
   - Use simple language. Avoid technical or complex terms.
   - Keep the tone respectful—never pressure the user.



— 


"""





from langchain_core.messages import AIMessage, HumanMessage

# def unregister_node(state: AgentState) -> AgentState:
#     print("🔐 unregister node...")

#     user_msg = state["messages"][-1].content.lower()
#     updated_messages = state["messages"]

#     if "yes" in user_msg or "register me" in user_msg:
        
#         updated_messages.append(AIMessage(content=f"✅ Registration started. Please fill the form: https://fak_form.com?"))
#         return {
#                 "messages": updated_messages
#     }


#     # Default LLM handling
#     response = llm_with_tools.invoke([prompt_unregister] + state["messages"])
#     return {
#         **state,
#         "messages": updated_messages + [response]
#     }



def unregister_node(state: AgentState) -> AgentState:
    print("🔐 unregister node...")

    user_msg = state["messages"][-1].content.lower()
    updated_messages = state["messages"]

    # ✅ User wants to register
    if "yes" in user_msg or "register me" in user_msg:
        updated_messages.append(AIMessage(content="✅ Registration started. Please fill the form: https://fak_form.com"))
        return {**state, "messages": updated_messages}
    
    # ✅ Check if message contains a possible ID
    if user_msg.isdigit():
        user_id = user_msg.strip()
        if check_user_id_in_csv(user_id):
            updated_messages.append(AIMessage(content="✅ Your registration is confirmed!"))
            return {
                "status": "registered",
                "user_id": user_id,
                "messages": updated_messages
            }
        else:
            updated_messages.append(AIMessage(content="❌ Couldn't find your ID or registration not completed."))
            return {**state, "messages": updated_messages}

    # 🤖 Default LLM fallback
    response = llm_with_tools.invoke([prompt_unregister] + state["messages"])
    return {
        **state,
        "messages": updated_messages + [response]
    }



def register_node(state: AgentState) -> AgentState:
    print("register node...")
    state["messages"].append(AIMessage(content="Welcome back! What would you like to do?"))
    return state


def categorizer_router(state: AgentState) -> str:
    return "register" if state["status"] == "registered" else "unregister"





In [107]:

builder = StateGraph(AgentState)

builder.add_node("categorizer", categorizer)
builder.add_node("unregister", unregister_node)
builder.add_node("register", register_node)
builder.add_node('tools', ToolNode(tools))

# Initial routing
builder.set_entry_point("categorizer")
builder.add_conditional_edges("categorizer", categorizer_router, {
    "register": "register",
    "unregister": "unregister"
})

# Tools condition (stays as is)
builder.add_conditional_edges('unregister', tools_condition)
builder.add_edge('tools', 'unregister')



# Register ends the flow
builder.add_edge("register", END)
builder.add_edge("unregister", END)

graph = builder.compile()


In [108]:
state = default_agent_state()
state["messages"].append(HumanMessage(content="hi"))

thread = {"configurable": {"thread_id": "1"}}

for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()
        print(state['status'], state.get('user_id'))

🔐 unregister node...
{'name': 'call_register_user', 'description': 'Call the FastAPI endpoint to register a new user.', 'parameters': {}}
{'name': 'call_confirm_registration', 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'parameters': {'type_': 6, 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'properties': {'user_id': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'properties': {}, 'required': []}}, 'required': ['user_id'], 'format_': '', 'nullable': False, 'enum': []}}
================================== Ai Message ==================================

Hi there! How can I help you today?
unregistered None


In [109]:

state["messages"].append(HumanMessage(content="i want to book ticket"))

thread = {"configurable": {"thread_id": "1"}}

for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()
        print(state['status'], state.get('user_id'))

User wants to book but is unregistered.
🔐 unregister node...
{'name': 'call_register_user', 'description': 'Call the FastAPI endpoint to register a new user.', 'parameters': {}}
{'name': 'call_confirm_registration', 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'parameters': {'type_': 6, 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'properties': {'user_id': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'properties': {}, 'required': []}}, 'required': ['user_id'], 'format_': '', 'nullable': False, 'enum': []}}
================================== Ai Message ==================================

I see you'd like to book a ticket.  To do that, you'll first need to register. Would you like me to share a registration link with you?
unregistered None


In [110]:

state["messages"].append(HumanMessage(content="yes"))

thread = {"configurable": {"thread_id": "1"}}

for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()
        print(state['status'], state.get('user_id'))

🔐 unregister node...
================================== Ai Message ==================================

✅ Registration started. Please fill the form: https://fak_form.com
unregistered None


In [111]:

state["messages"].append(HumanMessage(content="i have submit form"))

thread = {"configurable": {"thread_id": "1"}}

for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()
        print(state['status'], state.get('user_id'))

🔐 unregister node...
{'name': 'call_register_user', 'description': 'Call the FastAPI endpoint to register a new user.', 'parameters': {}}
{'name': 'call_confirm_registration', 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'parameters': {'type_': 6, 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'properties': {'user_id': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'properties': {}, 'required': []}}, 'required': ['user_id'], 'format_': '', 'nullable': False, 'enum': []}}
================================== Ai Message ==================================

Great!  Could you please provide me with your user ID so I can confirm your registration?
unregistered None


In [112]:

state["messages"].append(HumanMessage(content="its 123"))

thread = {"configurable": {"thread_id": "1"}}

for event in graph.stream(state, thread, stream_mode="values"):
    state = event
    last_msg = event["messages"][-1]
    if isinstance(last_msg, AIMessage):
        last_msg.pretty_print()
        print(state['status'], state.get('user_id'))

🔐 unregister node...
{'name': 'call_register_user', 'description': 'Call the FastAPI endpoint to register a new user.', 'parameters': {}}
{'name': 'call_confirm_registration', 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'parameters': {'type_': 6, 'description': "Call the FastAPI endpoint to confirm if user's account is created.", 'properties': {'user_id': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'properties': {}, 'required': []}}, 'required': ['user_id'], 'format_': '', 'nullable': False, 'enum': []}}
================================== Ai Message ==================================
Tool Calls:
  call_confirm_registration (3a9f3fb5-2b9b-47f3-87de-da1accbae862)
 Call ID: 3a9f3fb5-2b9b-47f3-87de-da1accbae862
  Args:
    user_id: 123
unregistered None
123
the confirmation tools working
🔐 unregister node...
{'name': 'call_register_user', 'description': 'Call the FastAPI endpoint to register a new user.', 'parameter

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Please ensure that the number of function response parts is equal to the number of function call parts of the function call turn.